In [2]:
# Standard libraries
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)
import logging
import argparse # Argument parsing

# Data manipulation and analysis
import pandas as pd
import numpy as np
import pickle
import torch
import torch.optim as optim
import copy                    # clone tensor
import time

# Custom imports
import GEMS_TCO
from GEMS_TCO import kernels
from GEMS_TCO import data_preprocess 
from GEMS_TCO import kernels 
from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import load_data
from GEMS_TCO import alg_optimization, alg_opt_Encoder
from GEMS_TCO import configuration as config

from typing import Optional, List, Tuple
from pathlib import Path
import typer
import json
from json import JSONEncoder

from GEMS_TCO.data_loader import load_data2

# square parameteriazation + stablize parametrization

In [4]:
import torch
import numpy as np
import torch.optim as optim
from scipy.spatial.distance import cdist
from torch.nn import Parameter

# --- 0. Global Parameters and Utility Functions ---
N_SPATIAL_POINTS = 1120
N_DAYS = 31
N_HOURS_PER_DAY = 8
N_FEATURES = 4
GRID_X = 40  
GRID_Y = 28  
LAT_MIN, LAT_MAX = 0, 5
LON_MIN, LON_MAX = 113, 123
BASE_DATE = '2024_07_y24m07day'

# --- 0. Global Parameters and Utility Functions ---
N_SPATIAL_POINTS = 1120
N_FEATURES = 4
# ... (rest of constants remain the same) ...

# Exponential Kernel Parameters (Targets)
SIGMA2_TRUE = 25.0 # TARGET Variance
RANGE_A_TRUE = 1    # TARGET Range
NUGGET_TRUE = 3.0   # TARGET Nugget for data generation

# Optimization Setup
ADAM_ITERATIONS = 500
ADAM_LEARNING_RATE = 0.01
LBFGS_MAX_STEPS = 50 
LBFGS_MAX_EVAL = 50 

OZONE_MEAN = 240.0

# --- COVARIANCE FUNCTIONS ---

def exponential_covariance_numpy(distances, sigma2, a, nugget):
    """Exponential covariance function (NumPy for Generation)."""
    # Cov(h) = sigma^2 * exp(-h/a)
    cov = sigma2 * np.exp(-distances / a)
    if distances.shape[0] == distances.shape[1]:
        cov[np.diag_indices_from(distances)] += (nugget + 1e-6)
    return cov

def exponential_covariance_torch(distances_torch, sigma2, a, nugget):
    """Exponential covariance function (PyTorch for Optimization)."""
    cov = sigma2 * torch.exp(-distances_torch / a)
    
    # Add nugget effect + jitter to the diagonal
    if distances_torch.shape[0] == distances_torch.shape[1]:
        jitter = 1e-6 
        diag_mask = torch.eye(cov.shape[0], device=cov.device)
        # Nugget is now a torch tensor parameter
        cov = cov + diag_mask * (nugget + jitter)
    return cov

# --- SHARED NLL Function using STABLE REPARAMETERIZATION ---
def neg_log_likelihood_torch_stable(raw_params, distances_torch, z_centered_torch):
    """
    Calculates -LL for PyTorch, optimizing Stable Reparameterization + log(nugget).
    raw_params[0] = raw_phi1_sqrt (for sigma2/a)
    raw_params[1] = raw_phi2_sqrt (for 1/a)
    raw_params[2] = log_nugget (for log(eta^2))
    """
    
    # 1. Apply Reparameterization and Positivity Constraints
    phi1 = raw_params[0].pow(2).squeeze() # Phi1 = sigma2 / a
    phi2 = raw_params[1].pow(2).squeeze() # Phi2 = 1 / a
    
    # Nugget is log-transformed for optimization
    nugget = torch.exp(raw_params[2]).squeeze() 
    
    epsilon = 1e-6
    
    # 2. Derive Original Parameters
    range_a = 1.0 / (phi2 + epsilon)          # Range: a = 1 / Phi2
    sigma2 = phi1 / (phi2 + epsilon)          # Variance: sigma2 = Phi1 / Phi2
    
    C = exponential_covariance_torch(distances_torch, sigma2, range_a, nugget)
    
    try:
        # Cholesky decomposition
        L = torch.linalg.cholesky(C)
        log_det = 2.0 * torch.sum(torch.log(torch.diag(L)))
        alpha = torch.linalg.solve(C, z_centered_torch.unsqueeze(1))
        quad_term = z_centered_torch.unsqueeze(0) @ alpha
        neg_LL = 0.5 * log_det + 0.5 * quad_term.squeeze()
        return neg_LL
    except RuntimeError:
        return torch.tensor(1e15, device=C.device)

# --- Data Generation Function (Unchanged) ---
def generate_ozone_data_map(coords, sigma2, a, nugget, mean, time_index):
    n_points = coords.shape[0]
    distances = cdist(coords, coords, metric='euclidean')
    Cov = exponential_covariance_numpy(distances, sigma2, a, nugget) 
    Cov = (Cov + Cov.T) / 2
    
    try:
        L = np.linalg.cholesky(Cov)
    except np.linalg.LinAlgError:
        return np.zeros((n_points, N_FEATURES))

    W = np.random.normal(0, 1, size=(n_points, 1))
    Z_correlated = L @ W
    ozone_values = mean + Z_correlated
    
    data_np = np.zeros((n_points, N_FEATURES))
    data_np[:, 0:1] = ozone_values             
    data_np[:, 1] = coords[:, 1] * 10 + 2      
    data_np[:, 2] = coords[:, 0] * 40 + 250    
    data_np[:, 3] = time_index                 
    return data_np


# --- 1. Data Generation Execution ---
df_day_aggregated_list = []
print("--- Starting Data Generation ---")
lat_coords = np.linspace(LAT_MIN, LAT_MAX, GRID_Y)
lon_coords = np.linspace(LON_MIN, LON_MAX, GRID_X)
coords_latlon = np.array([[lat, lon] for lat in lat_coords for lon in lon_coords])

# Generate only one hour of data for fitting the spatial model
data_np = generate_ozone_data_map(
    coords_latlon, SIGMA2_TRUE, RANGE_A_TRUE, NUGGET_TRUE, OZONE_MEAN, 21.0
)
df_day_aggregated_list.append(torch.tensor(data_np, dtype=torch.float))

print("--- Data Generation Complete ---")

# --- 2. Data Preparation ---
data_to_fit = df_day_aggregated_list[0][:N_SPATIAL_POINTS, :] 
z_data = data_to_fit[:, 0].numpy()
coordinates = coords_latlon[:, [1, 0]] 
distances_np = cdist(coordinates, coordinates, metric='euclidean')
z_centered_np = z_data - np.mean(z_data)

# Convert to Torch Tensors
distances_torch = torch.tensor(distances_np, dtype=torch.float)
z_centered_torch = torch.tensor(z_centered_np, dtype=torch.float)

# --- Initial Parameter Setup (Shared) ---
PHI1_TARGET = SIGMA2_TRUE / RANGE_A_TRUE
PHI2_TARGET = 1.0 / RANGE_A_TRUE

raw_phi1_sqrt_start = np.sqrt(PHI1_TARGET + 4.0) 
raw_phi2_sqrt_start = np.sqrt(PHI2_TARGET + 0.133) 

# 💥 NEW: Initialize nugget parameter (log scale) to 0.3
NUGGET_INIT = 0.3
LOG_NUGGET_START = np.log(NUGGET_INIT) 

initial_params_stable = [raw_phi1_sqrt_start, raw_phi2_sqrt_start, LOG_NUGGET_START]


# ----------------------------------------------------
# A. Optimization with L-BFGS (PyTorch) - STABLE
# ----------------------------------------------------

# Reset parameters for LBFGS
# 💥 NEW: raw_params_lbfgs is a tensor of 3 parameters
raw_params_lbfgs = torch.tensor(
    initial_params_stable, 
    dtype=torch.float, 
    requires_grad=True
)

optimizer_lbfgs = optim.LBFGS(
    [raw_params_lbfgs], 
    lr=1.0, 
    max_iter=LBFGS_MAX_STEPS,
    max_eval=LBFGS_MAX_EVAL 
)

final_loss_lbfgs = torch.tensor(0.0)
print("\n--- A. Starting MLE Optimization (PyTorch L-BFGS) - STABLE ---")

# L-BFGS requires a "closure" function
def closure_lbfgs():
    optimizer_lbfgs.zero_grad()
    # 💥 NEW: Call NLL without the fixed_nugget argument
    loss = neg_log_likelihood_torch_stable(raw_params_lbfgs, distances_torch, z_centered_torch)
    if not torch.isinf(loss) and not torch.isnan(loss):
        loss.backward()
    return loss

# L-BFGS Optimization Loop
for step in range(LBFGS_MAX_STEPS):
    loss = optimizer_lbfgs.step(closure_lbfgs)
    final_loss_lbfgs = loss
    
    if (step + 1) % 5 == 0: 
        phi1 = raw_params_lbfgs[0].pow(2).item()
        phi2 = raw_params_lbfgs[1].pow(2).item()
        current_nugget = torch.exp(raw_params_lbfgs[2]).item() # 💥 NEW: Get current nugget
        current_sigma2 = phi1 / (phi2 + 1e-6)
        current_a = 1.0 / (phi2 + 1e-6)
        
        grad_phi1 = raw_params_lbfgs.grad[0].item() if raw_params_lbfgs.grad is not None else 0.0
        grad_phi2 = raw_params_lbfgs.grad[1].item() if raw_params_lbfgs.grad is not None else 0.0
        grad_nugget = raw_params_lbfgs.grad[2].item() if raw_params_lbfgs.grad is not None else 0.0
        
        print(f"LBFGS Step {step + 1}/{LBFGS_MAX_STEPS}, NLL: {loss.item():.2f}, Params: [σ²: {current_sigma2:.3f}, a: {current_a:.3f}, η²: {current_nugget:.3f}], Grads: [Φ1_raw: {grad_phi1:.4f}, Φ2_raw: {grad_phi2:.4f}, log(η²)_raw: {grad_nugget:.4f}]")


# ----------------------------------------------------
# B. Optimization with Adam (PyTorch) - STABLE
# ----------------------------------------------------

# Reset parameters for Adam (Use the same start point)
# 💥 NEW: raw_params_adam is a tensor of 3 parameters
raw_params_adam = torch.tensor(
    initial_params_stable, 
    dtype=torch.float, 
    requires_grad=True
)

optimizer_adam = optim.Adam(
    [raw_params_adam], 
    lr=ADAM_LEARNING_RATE
)

final_loss_adam = torch.tensor(0.0)
print(f"\n--- B. Starting MLE Optimization (PyTorch Adam) - STABLE ---")

# Adam Optimization Loop
for epoch in range(ADAM_ITERATIONS):
    optimizer_adam.zero_grad()
    
    # 💥 NEW: Call NLL without the fixed_nugget argument
    loss = neg_log_likelihood_torch_stable(raw_params_adam, distances_torch, z_centered_torch)
    
    if torch.isinf(loss) or torch.isnan(loss):
        loss = torch.tensor(1e15, device=loss.device)
        break

    loss.backward()
    optimizer_adam.step()
    final_loss_adam = loss
    
    if (epoch + 1) % 50 == 0: 
        phi1 = raw_params_adam[0].pow(2).item()
        phi2 = raw_params_adam[1].pow(2).item()
        current_nugget = torch.exp(raw_params_adam[2]).item() # 💥 NEW: Get current nugget
        current_sigma2 = phi1 / (phi2 + 1e-6)
        current_a = 1.0 / (phi2 + 1e-6)
        
        grad_phi1 = raw_params_adam.grad[0].item() if raw_params_adam.grad is not None else 0.0
        grad_phi2 = raw_params_adam.grad[1].item() if raw_params_adam.grad is not None else 0.0
        grad_nugget = raw_params_adam.grad[2].item() if raw_params_adam.grad is not None else 0.0
        
        print(f"Adam Epoch {epoch + 1}/{ADAM_ITERATIONS}, NLL: {loss.item():.2f}, Params: [σ²: {current_sigma2:.3f}, a: {current_a:.3f}, η²: {current_nugget:.3f}], Grads: [Φ1_raw: {grad_phi1:.4f}, Φ2_raw: {grad_phi2:.4f}, log(η²)_raw: {grad_nugget:.4f}]")


# ----------------------------------------------------
# 3. Display Results
# ----------------------------------------------------
print("\n" + "="*70)
print(f"TARGET PARAMETERS: Variance (σ²)={SIGMA2_TRUE}, Range (a)={RANGE_A_TRUE}, Nugget (η²)={NUGGET_TRUE}")
print(f"INITIAL NUGGET GUESS: {NUGGET_INIT}")
print("="*70)

# --- L-BFGS Results ---
phi1_lbfgs = raw_params_lbfgs[0].pow(2).detach().numpy().item()
phi2_lbfgs = raw_params_lbfgs[1].pow(2).detach().numpy().item()
log_nugget_lbfgs = raw_params_lbfgs[2].detach().numpy().item() # 💥 NEW
fitted_nugget_lbfgs = np.exp(log_nugget_lbfgs)                 # 💥 NEW

fitted_sigma2_lbfgs = phi1_lbfgs / (phi2_lbfgs + 1e-6)
fitted_range_a_lbfgs = 1.0 / (phi2_lbfgs + 1e-6)

print("✨ PyTorch L-BFGS Results (Stable Reparameterization):")
print(f"  * Fitted Variance (σ²): {fitted_sigma2_lbfgs:.3f} (Target: {SIGMA2_TRUE})")
print(f"  * Fitted Range (a): {fitted_range_a_lbfgs:.3f} (Target: {RANGE_A_TRUE})")
print(f"  * Fitted Nugget (η²): {fitted_nugget_lbfgs:.3f} (Target: {NUGGET_TRUE})") # 💥 NEW
print(f"  * Final -LL Value: {final_loss_lbfgs.item():.2f}")
print(f"  * Optimization Steps: {LBFGS_MAX_STEPS} steps")

# --- Adam Results ---
phi1_adam = raw_params_adam[0].pow(2).detach().numpy().item()
phi2_adam = raw_params_adam[1].pow(2).detach().numpy().item()
log_nugget_adam = raw_params_adam[2].detach().numpy().item() # 💥 NEW
fitted_nugget_adam = np.exp(log_nugget_adam)                 # 💥 NEW

fitted_sigma2_adam = phi1_adam / (phi2_adam + 1e-6)
fitted_range_a_adam = 1.0 / (phi2_adam + 1e-6)

print("\n🚀 PyTorch Adam Results (Stable Reparameterization):")
print(f"  * Fitted Variance (σ²): {fitted_sigma2_adam:.3f} (Target: {SIGMA2_TRUE})")
print(f"  * Fitted Range (a): {fitted_range_a_adam:.3f} (Target: {RANGE_A_TRUE})")
print(f"  * Fitted Nugget (η²): {fitted_nugget_adam:.3f} (Target: {NUGGET_TRUE})") # 💥 NEW
print(f"  * Final -LL Value: {final_loss_adam.item():.2f}")
print(f"  * Optimization Steps: {ADAM_ITERATIONS} epochs")
print("="*70)

--- Starting Data Generation ---
--- Data Generation Complete ---

--- A. Starting MLE Optimization (PyTorch L-BFGS) - STABLE ---
LBFGS Step 5/50, NLL: 1791.75, Params: [σ²: 19.998, a: 0.875, η²: 2.841], Grads: [Φ1_raw: -0.0001, Φ2_raw: -0.0001, log(η²)_raw: -0.0002]
LBFGS Step 10/50, NLL: 1791.75, Params: [σ²: 19.998, a: 0.875, η²: 2.841], Grads: [Φ1_raw: -0.0001, Φ2_raw: -0.0001, log(η²)_raw: -0.0002]
LBFGS Step 15/50, NLL: 1791.75, Params: [σ²: 19.998, a: 0.875, η²: 2.841], Grads: [Φ1_raw: -0.0001, Φ2_raw: -0.0001, log(η²)_raw: -0.0002]
LBFGS Step 20/50, NLL: 1791.75, Params: [σ²: 19.998, a: 0.875, η²: 2.841], Grads: [Φ1_raw: -0.0001, Φ2_raw: -0.0001, log(η²)_raw: -0.0002]
LBFGS Step 25/50, NLL: 1791.75, Params: [σ²: 19.998, a: 0.875, η²: 2.841], Grads: [Φ1_raw: -0.0001, Φ2_raw: -0.0001, log(η²)_raw: -0.0002]
LBFGS Step 30/50, NLL: 1791.75, Params: [σ²: 19.998, a: 0.875, η²: 2.841], Grads: [Φ1_raw: -0.0001, Φ2_raw: -0.0001, log(η²)_raw: -0.0002]
LBFGS Step 35/50, NLL: 1791.75, Param

# test 11/10/25

# L BFGS vs Adams  1120 reparametrization for anisotrpy

In [10]:
import torch
import numpy as np
import torch.optim as optim
from scipy.spatial.distance import cdist
from torch.nn import Parameter
import time

# --- 0. Global Parameters and Utility Functions ---
# 💥 MODIFIED: Increased grid density
N_SPATIAL_POINTS = 4480  # 80 * 56
GRID_X = 80              # 💥 MODIFIED
GRID_Y = 56              # 💥 MODIFIED

N_DAYS = 31
N_HOURS_PER_DAY = 8
N_FEATURES = 4
LAT_MIN, LAT_MAX = 0, 5
LON_MIN, LON_MAX = 113, 123
BASE_DATE = '2024_07_y24m07day' 

# Exponential Kernel Parameters (Targets)
SIGMA2_TRUE = 11.0      # TARGET Variance
RANGE_A_TRUE = 1.9      # TARGET Range
ANISOTROPY_RATIO_TRUE = 0.5 # TARGET Anisotropy
PHI3_TARGET_SQ = ANISOTROPY_RATIO_TRUE**2 
NUGGET_TRUE = 0.3       # TARGET Nugget for data generation

# --- Simulation Controls ---
NUM_SIMULATIONS = 1      
PRINT_EPOCHS = False      # Set to False to silence epoch logging

# Optimization Setup
ADAM_ITERATIONS = 500
ADAM_LEARNING_RATE = 0.01

# L-BFGS Setup
LBFGS_MAX_STEPS = 50 
LBFGS_MAX_EVAL = 50 

OZONE_MEAN = 240.0

# --- COVARIANCE FUNCTIONS ---

def exponential_covariance_numpy(distances, sigma2, a, nugget):
    """Exponential covariance function (NumPy for Generation)."""
    cov = sigma2 * np.exp(-distances / a)
    if distances.shape[0] == distances.shape[1]:
        cov[np.diag_indices_from(distances)] += (nugget + 1e-6)
    return cov

def exponential_covariance_torch(distances_torch, sigma2, a, nugget):
    """Exponential covariance function (PyTorch for Optimization)."""
    cov = sigma2 * torch.exp(-distances_torch / a)
    
    if distances_torch.shape[0] == distances_torch.shape[1]:
        jitter = 1e-6 
        diag_mask = torch.eye(cov.shape[0], device=cov.device)
        cov = cov + diag_mask * (nugget + jitter)
    return cov

# --- NLL Function (4 parameters) ---
def neg_log_likelihood_torch_stable(raw_params, d_lon_sq_torch, d_lat_sq_torch, z_centered_torch):
    """
    Calculates -LL for PyTorch (optimizing Stable Reparameterization + log(nugget)).
    """
    
    epsilon = 1e-6
    
    # 1. Apply Reparameterization
    phi1 = raw_params[0].pow(2).squeeze() + epsilon # theta_1 = sigma2 / a
    phi2 = raw_params[1].pow(2).squeeze() + epsilon # theta_2 = 1 / a
    phi3 = raw_params[2].pow(2).squeeze() + epsilon # phi_3 = theta_3^2
    nugget = torch.exp(raw_params[3]).squeeze()     # Nugget = exp(log_nugget)
    
    # 2. Derive Original Parameters
    range_a = 1.0 / phi2          
    sigma2 = phi1 / phi2          
    
    # 3. Compute Anisotropic Distance
    aniso_dist_sq = (d_lon_sq_torch / phi3) + d_lat_sq_torch
    aniso_dist = torch.sqrt(aniso_dist_sq + epsilon)
    
    # 4. Calculate Covariance Matrix C
    C = exponential_covariance_torch(aniso_dist, sigma2, range_a, nugget)
    
    try:
        L = torch.linalg.cholesky(C)
        log_det = 2.0 * torch.sum(torch.log(torch.diag(L)))
        alpha = torch.linalg.solve(C, z_centered_torch.unsqueeze(1))
        quad_term = z_centered_torch.unsqueeze(0) @ alpha
        neg_LL = 0.5 * log_det + 0.5 * quad_term.squeeze()
        
        if torch.isnan(neg_LL) or torch.isinf(neg_LL):
            return torch.tensor(1e15, device=C.device, dtype=torch.float) + raw_params.sum() * 0.0

        return neg_LL
    except RuntimeError: # Catch Cholesky failures
        return torch.tensor(1e15, device=C.device, dtype=torch.float) + raw_params.sum() * 0.0


# --- Data Generation Function ---
def generate_ozone_data_map(coords, sigma2, a, nugget, mean, time_index, anisotropy_ratio):
    n_points = coords.shape[0]
    coords_transformed = coords.copy()
    coords_transformed[:, 1] = coords_transformed[:, 1] / anisotropy_ratio
    
    distances = cdist(coords_transformed, coords_transformed, metric='euclidean')
    
    Cov = exponential_covariance_numpy(distances, sigma2, a, nugget) 
    Cov = (Cov + Cov.T) / 2
    
    try:
        L = np.linalg.cholesky(Cov)
    except np.linalg.LinAlgError:
        print(f"Warning: Cholesky failed in data generation with N={n_points}. Cov matrix may be singular.")
        return np.zeros((n_points, N_FEATURES))

    W = np.random.normal(0, 1, size=(n_points, 1))
    Z_correlated = L @ W
    ozone_values = mean + Z_correlated
    
    data_np = np.zeros((n_points, N_FEATURES))
    data_np[:, 0:1] = ozone_values             
    data_np[:, 1] = coords[:, 1] * 10 + 2      # Original lon
    data_np[:, 2] = coords[:, 0] * 40 + 250    # Original lat
    data_np[:, 3] = time_index                 
    return data_np


# ==========================================================
# --- MAIN SIMULATION LOOP ---
# ==========================================================
if __name__ == '__main__':

    # --- 1. Setup Simulation ---
    start_time_total = time.time()
    print(f"--- Starting {NUM_SIMULATIONS} Simulation Runs ---")
    print(f"--- Grid Size: {GRID_X}x{GRID_Y} (N = {N_SPATIAL_POINTS}) ---")
    
    # Pre-calculate coordinate geometry (this doesn't change)
    lat_coords = np.linspace(LAT_MIN, LAT_MAX, GRID_Y)
    lon_coords = np.linspace(LON_MIN, LON_MAX, GRID_X)
    coords_latlon = np.array([[lat, lon] for lat in lat_coords for lon in lon_coords]) # [lat, lon]
    
    coordinates = coords_latlon[:, [1, 0]] # Switch to [lon, lat]
    lons = coordinates[:, 0:1] 
    lats = coordinates[:, 1:2] 
    d_lon_np = cdist(lons, lons, metric='euclidean')
    d_lat_np = cdist(lats, lats, metric='euclidean')
    d_lon_sq_np = np.square(d_lon_np)
    d_lat_sq_np = np.square(d_lat_np)
    
    d_lon_sq_torch = torch.tensor(d_lon_sq_np, dtype=torch.float)
    d_lat_sq_torch = torch.tensor(d_lat_sq_np, dtype=torch.float)
    
    # Calculate initial parameter guesses (these are reset every loop)
    PHI1_TARGET = SIGMA2_TRUE / RANGE_A_TRUE
    PHI2_TARGET = 1.0 / RANGE_A_TRUE
    PHI3_TARGET = PHI3_TARGET_SQ
    
    raw_phi1_sqrt_start = np.sqrt(PHI1_TARGET - 3.0) 
    raw_phi2_sqrt_start = np.sqrt(PHI2_TARGET - 0.1) 
    raw_phi3_sqrt_start = np.sqrt(3.0) 
    NUGGET_INIT_GUESS = 0.3
    LOG_NUGGET_START = np.log(NUGGET_INIT_GUESS)

    initial_params_stable = [
        raw_phi1_sqrt_start, 
        raw_phi2_sqrt_start,
        raw_phi3_sqrt_start,
        LOG_NUGGET_START
    ]
    
    # --- 2. Initialize Result Storage ---
    results_lbfgs = []
    results_adam = []

    # --- 3. Run Simulation Loop ---
    for i in range(NUM_SIMULATIONS):
        run_start_time = time.time()
        print(f"--- Running Simulation {i+1}/{NUM_SIMULATIONS} ---")

        # --- A. Generate NEW Data ---
        data_np = generate_ozone_data_map(
            coords_latlon, SIGMA2_TRUE, RANGE_A_TRUE, NUGGET_TRUE, OZONE_MEAN, 21.0,
            ANISOTROPY_RATIO_TRUE
        )
        data_to_fit = torch.tensor(data_np, dtype=torch.float)[:N_SPATIAL_POINTS, :]
        z_data = data_to_fit[:, 0].numpy()
        z_centered_np = z_data - np.mean(z_data)
        z_centered_torch = torch.tensor(z_centered_np, dtype=torch.float)

        # --- B. Run L-BFGS ---
        raw_params_lbfgs = torch.tensor(
            initial_params_stable, 
            dtype=torch.float, 
            requires_grad=True
        )
        optimizer_lbfgs = optim.LBFGS(
            [raw_params_lbfgs], 
            lr=1.0, 
            max_iter=LBFGS_MAX_STEPS,
            max_eval=LBFGS_MAX_EVAL 
        )
        final_loss_lbfgs = torch.tensor(0.0)

        def closure_lbfgs():
            optimizer_lbfgs.zero_grad()
            loss = neg_log_likelihood_torch_stable(
                raw_params_lbfgs, d_lon_sq_torch, d_lat_sq_torch, z_centered_torch
            )
            if not torch.isinf(loss) and not torch.isnan(loss):
                loss.backward()
            return loss

        for step in range(LBFGS_MAX_STEPS):
            loss = optimizer_lbfgs.step(closure_lbfgs)
            final_loss_lbfgs = loss
            
            if PRINT_EPOCHS and ((step + 1) % 5 == 0): 
                # (Logging is silenced by default)
                pass 

        # --- C. Run Adam ---
        raw_params_adam = torch.tensor(
            initial_params_stable, 
            dtype=torch.float, 
            requires_grad=True
        )
        optimizer_adam = optim.Adam([raw_params_adam], lr=ADAM_LEARNING_RATE)
        final_loss_adam = torch.tensor(0.0)

        for epoch in range(ADAM_ITERATIONS):
            optimizer_adam.zero_grad()
            loss = neg_log_likelihood_torch_stable(
                raw_params_adam, d_lon_sq_torch, d_lat_sq_torch, z_centered_torch
            )
            if torch.isinf(loss) or torch.isnan(loss):
                continue 
            loss.backward()
            optimizer_adam.step()
            final_loss_adam = loss
            
            if PRINT_EPOCHS and ((epoch + 1) % 50 == 0): 
                # (Logging is silenced by default)
                pass
        
        # --- D. Store Results for this run ---
        
        # L-BFGS Results
        with torch.no_grad():
            phi1_lbfgs = raw_params_lbfgs[0].pow(2).item()
            phi2_lbfgs = raw_params_lbfgs[1].pow(2).item()
            phi3_lbfgs = raw_params_lbfgs[2].pow(2).item()
            fitted_nugget_lbfgs = torch.exp(raw_params_lbfgs[3]).item()
        
        results_lbfgs.append({
            'sigma2': phi1_lbfgs / (phi2_lbfgs + 1e-6),
            'range_a': 1.0 / (phi2_lbfgs + 1e-6),
            'ratio': np.sqrt(phi3_lbfgs),
            'nugget': fitted_nugget_lbfgs,
            'nll': final_loss_lbfgs.item()
        })
        
        # Adam Results
        with torch.no_grad():
            phi1_adam = raw_params_adam[0].pow(2).item()
            phi2_adam = raw_params_adam[1].pow(2).item()
            phi3_adam = raw_params_adam[2].pow(2).item()
            fitted_nugget_adam = torch.exp(raw_params_adam[3]).item()
        
        results_adam.append({
            'sigma2': phi1_adam / (phi2_adam + 1e-6),
            'range_a': 1.0 / (phi2_adam + 1e-6),
            'ratio': np.sqrt(phi3_adam),
            'nugget': fitted_nugget_adam,
            'nll': final_loss_adam.item()
        })
        
        run_end_time = time.time()
        print(f"--- Simulation {i+1} complete. Time elapsed: {run_end_time - run_start_time:.2f}s ---")


    print("\n--- All Simulations Complete ---")
    end_time_total = time.time()

    # ==========================================================
    # --- 4. Display Aggregate Results ---
    # ==========================================================
    
    print("\n" + "="*75)
    print(f"TARGET PARAMETERS: Variance (σ²)={SIGMA2_TRUE}, Range (a)={RANGE_A_TRUE}, Anisotropy (θ₃-ratio)={ANISOTROPY_RATIO_TRUE}, Nugget (η²)={NUGGET_TRUE}")
    print(f"AGGREGATE RESULTS OVER {NUM_SIMULATIONS} RUNS")
    print(f"Grid Size: {GRID_X}x{GRID_Y} (N = {N_SPATIAL_POINTS})")
    print(f"Total time: {end_time_total - start_time_total:.2f} seconds")
    print("="*75)

    # Helper function to calculate and print stats
    def print_stats(results_list, optimizer_name):
        # Convert list of dicts to a dict of numpy arrays
        # Filter out NaN/Inf values that could arise from failed runs
        params = {}
        for key in results_list[0].keys():
            valid_values = [res[key] for res in results_list if np.isfinite(res[key])]
            if not valid_values:
                valid_values = [np.nan] # Handle case where all runs failed for a param
            params[key] = np.array(valid_values)
        
        num_valid = len(params['nll'])
        num_total = len(results_list)

        print(f"✨ {optimizer_name} Average Results ({num_valid}/{num_total} Valid Runs):")
        
        # Helper for printing mean/std
        def ms(key):
            return f"{np.mean(params[key]):.3f} (Std: {np.std(params[key]):.3f})"

        print(f"  * Fitted Variance (σ²): {ms('sigma2')} (Target: {SIGMA2_TRUE})")
        print(f"  * Fitted Range (a): {ms('range_a')} (Target: {RANGE_A_TRUE})")
        print(f"  * Fitted Anisotropy (θ₃-ratio): {ms('ratio')} (Target: {ANISOTROPY_RATIO_TRUE})")
        print(f"  * Fitted Nugget (η²): {ms('nugget')} (Target: {NUGGET_TRUE})")
        print(f"  * Final -LL Value: {np.mean(params['nll']):.2f} (Std: {np.std(params['nll']):.2f})")

    try:
        if results_lbfgs:
            print_stats(results_lbfgs, "PyTorch L-BFGS")
        else:
            print("No valid L-BFGS results to display.")
            
        print("\n" + "-"*75 + "\n") # Add a separator
        
        if results_adam:
            print_stats(results_adam, "PyTorch Adam")
        else:
            print("No valid Adam results to display.")

    except Exception as e:
        print(f"An error occurred during final reporting: {e}")
        print("\nRaw L-BFGS Results:", results_lbfgs)
        print("\nRaw Adam Results:", results_adam)

    print("="*75)

--- Starting 1 Simulation Runs ---
--- Grid Size: 80x56 (N = 4480) ---
--- Running Simulation 1/1 ---
--- Simulation 1 complete. Time elapsed: 248.29s ---

--- All Simulations Complete ---

TARGET PARAMETERS: Variance (σ²)=11.0, Range (a)=1.9, Anisotropy (θ₃-ratio)=0.5, Nugget (η²)=0.3
AGGREGATE RESULTS OVER 1 RUNS
Grid Size: 80x56 (N = 4480)
Total time: 248.37 seconds
✨ PyTorch L-BFGS Average Results (1/1 Valid Runs):
  * Fitted Variance (σ²): 10.539 (Std: 0.000) (Target: 11.0)
  * Fitted Range (a): 1.702 (Std: 0.000) (Target: 1.9)
  * Fitted Anisotropy (θ₃-ratio): 0.491 (Std: 0.000) (Target: 0.5)
  * Fitted Nugget (η²): 0.244 (Std: 0.000) (Target: 0.3)
  * Final -LL Value: 2770.48 (Std: 0.00)

---------------------------------------------------------------------------

✨ PyTorch Adam Average Results (1/1 Valid Runs):
  * Fitted Variance (σ²): 11.066 (Std: 0.000) (Target: 11.0)
  * Fitted Range (a): 1.980 (Std: 0.000) (Target: 1.9)
  * Fitted Anisotropy (θ₃-ratio): 0.472 (Std: 0.000) 

use log transformation instead of optimizing raw prams .pow()     torch.exp(raw_log_param) this guarantees positivity without epsilon hack and optimization more lobust

# log + reparam + non zero nugget

In [19]:
import torch
import numpy as np
import torch.optim as optim
from scipy.spatial.distance import cdist
from torch.nn import Parameter
import time

# --- 0. Global Parameters and Utility Functions ---
# 💥 MODIFIED: Increased grid density
N_SPATIAL_POINTS = 4480  # 80 * 56
GRID_X = 80              # 💥 MODIFIED
GRID_Y = 56              # 💥 MODIFIED

N_DAYS = 31
N_HOURS_PER_DAY = 8
N_FEATURES = 4
LAT_MIN, LAT_MAX = 0, 5
LON_MIN, LON_MAX = 113, 123
BASE_DATE = '2024_07_y24m07day' 

# Exponential Kernel Parameters (Targets)
SIGMA2_TRUE = 11.0      # TARGET Variance
RANGE_A_TRUE = 1.9      # TARGET Range
ANISOTROPY_RATIO_TRUE = 0.5 # TARGET Anisotropy
PHI3_TARGET_SQ = ANISOTROPY_RATIO_TRUE**2 
NUGGET_TRUE = 0.3       # TARGET Nugget for data generation

# --- Simulation Controls ---
NUM_SIMULATIONS = 1      
PRINT_EPOCHS = False      # Set to False to silence epoch logging

# Optimization Setup
ADAM_ITERATIONS = 500
ADAM_LEARNING_RATE = 0.01

# L-BFGS Setup
LBFGS_MAX_STEPS = 50 
LBFGS_MAX_EVAL = 50 

OZONE_MEAN = 240.0

# --- COVARIANCE FUNCTIONS ---

def exponential_covariance_numpy(distances, sigma2, a, nugget):
    """Exponential covariance function (NumPy for Generation)."""
    cov = sigma2 * np.exp(-distances / a)
    if distances.shape[0] == distances.shape[1]:
        cov[np.diag_indices_from(distances)] += (nugget + 1e-6)
    return cov

def exponential_covariance_torch(distances_torch, sigma2, a, nugget):
    """Exponential covariance function (PyTorch for Optimization)."""
    cov = sigma2 * torch.exp(-distances_torch / a)
    
    if distances_torch.shape[0] == distances_torch.shape[1]:
        jitter = 1e-6 
        diag_mask = torch.eye(cov.shape[0], device=cov.device)
        cov = cov + diag_mask * (nugget + jitter)
    return cov

# --- 💥 MODIFIED NLL Function (Log-Frame) 💥 ---
def neg_log_likelihood_torch_stable(raw_params, d_lon_sq_torch, d_lat_sq_torch, z_centered_torch):
    """
    Calculates -LL for PyTorch (optimizing Log-Reparameterization).
    """
    
    epsilon = 1e-6 # Retain epsilon for sqrt and division, but not for exp
    
    # 1. Apply Log-Reparameterization
    phi1   = torch.exp(raw_params[0]).squeeze() # theta_1 = sigma2 / a
    phi2   = torch.exp(raw_params[1]).squeeze() # theta_2 = 1 / a
    phi3   = torch.exp(raw_params[2]).squeeze() # phi_3 = theta_3^2
    nugget = torch.exp(raw_params[3]).squeeze() # Nugget = exp(log_nugget)
    
    # 2. Derive Original Parameters
    range_a = 1.0 / (phi2 + epsilon) # Add epsilon to prevent 1/0
    sigma2 = phi1 / (phi2 + epsilon)
    
    # 3. Compute Anisotropic Distance
    aniso_dist_sq = (d_lon_sq_torch / (phi3 + epsilon)) + d_lat_sq_torch # Add epsilon
    aniso_dist = torch.sqrt(aniso_dist_sq + epsilon)
    
    # 4. Calculate Covariance Matrix C
    C = exponential_covariance_torch(aniso_dist, sigma2, range_a, nugget)
    
    try:
        L = torch.linalg.cholesky(C)
        log_det = 2.0 * torch.sum(torch.log(torch.diag(L)))
        alpha = torch.linalg.solve(C, z_centered_torch.unsqueeze(1))
        quad_term = z_centered_torch.unsqueeze(0) @ alpha
        neg_LL = 0.5 * log_det + 0.5 * quad_term.squeeze()
        
        if torch.isnan(neg_LL) or torch.isinf(neg_LL):
            return torch.tensor(1e15, device=C.device, dtype=torch.float) + raw_params.sum() * 0.0

        return neg_LL
    except RuntimeError: # Catch Cholesky failures
        return torch.tensor(1e15, device=C.device, dtype=torch.float) + raw_params.sum() * 0.0


# --- Data Generation Function ---
def generate_ozone_data_map(coords, sigma2, a, nugget, mean, time_index, anisotropy_ratio):
    n_points = coords.shape[0]
    coords_transformed = coords.copy()
    coords_transformed[:, 1] = coords_transformed[:, 1] / anisotropy_ratio
    
    distances = cdist(coords_transformed, coords_transformed, metric='euclidean')
    
    Cov = exponential_covariance_numpy(distances, sigma2, a, nugget) 
    Cov = (Cov + Cov.T) / 2
    
    try:
        L = np.linalg.cholesky(Cov)
    except np.linalg.LinAlgError:
        print(f"Warning: Cholesky failed in data generation with N={n_points}. Cov matrix may be singular.")
        return np.zeros((n_points, N_FEATURES))

    W = np.random.normal(0, 1, size=(n_points, 1))
    Z_correlated = L @ W
    ozone_values = mean + Z_correlated
    
    data_np = np.zeros((n_points, N_FEATURES))
    data_np[:, 0:1] = ozone_values             
    data_np[:, 1] = coords[:, 1] * 10 + 2      # Original lon
    data_np[:, 2] = coords[:, 0] * 40 + 250    # Original lat
    data_np[:, 3] = time_index                 
    return data_np


# ==========================================================
# --- MAIN SIMULATION LOOP ---
# ==========================================================
if __name__ == '__main__':

    # --- 1. Setup Simulation ---
    start_time_total = time.time()
    print(f"--- Starting {NUM_SIMULATIONS} Simulation Runs ---")
    print(f"--- Grid Size: {GRID_X}x{GRID_Y} (N = {N_SPATIAL_POINTS}) ---")
    
    # Pre-calculate coordinate geometry (this doesn't change)
    lat_coords = np.linspace(LAT_MIN, LAT_MAX, GRID_Y)
    lon_coords = np.linspace(LON_MIN, LON_MAX, GRID_X)
    coords_latlon = np.array([[lat, lon] for lat in lat_coords for lon in lon_coords]) # [lat, lon]
    
    coordinates = coords_latlon[:, [1, 0]] # Switch to [lon, lat]
    lons = coordinates[:, 0:1] 
    lats = coordinates[:, 1:2] 
    d_lon_np = cdist(lons, lons, metric='euclidean')
    d_lat_np = cdist(lats, lats, metric='euclidean')
    d_lon_sq_np = np.square(d_lon_np)
    d_lat_sq_np = np.square(d_lat_np)
    
    d_lon_sq_torch = torch.tensor(d_lon_sq_np, dtype=torch.float)
    d_lat_sq_torch = torch.tensor(d_lat_sq_np, dtype=torch.float)
    
    # Calculate initial parameter guesses (these are reset every loop)
    PHI1_TARGET = SIGMA2_TRUE / RANGE_A_TRUE
    PHI2_TARGET = 1.0 / RANGE_A_TRUE
    PHI3_TARGET = PHI3_TARGET_SQ
    
    # --- 💥 MODIFIED: Initial Guesses are now in Log-Frame 💥 ---
    # Use log of the (off-target) values
    raw_log_phi1_start = np.log(max(PHI1_TARGET - 3.0, 1e-6))
    raw_log_phi2_start = np.log(max(PHI2_TARGET - 0.1, 1e-6))
    raw_log_phi3_start = np.log(max(3.0, 1e-6)) # Start at 3.0
    NUGGET_INIT_GUESS = 0.3
    LOG_NUGGET_START = np.log(NUGGET_INIT_GUESS)

    initial_params_stable = [
        raw_log_phi1_start, 
        raw_log_phi2_start,
        raw_log_phi3_start,
        LOG_NUGGET_START
    ]
    
    # --- 2. Initialize Result Storage ---
    results_lbfgs = []
    results_adam = []

    # --- 3. Run Simulation Loop ---
    for i in range(NUM_SIMULATIONS):
        run_start_time = time.time()
        print(f"--- Running Simulation {i+1}/{NUM_SIMULATIONS} ---")

        # --- A. Generate NEW Data ---
        data_np = generate_ozone_data_map(
            coords_latlon, SIGMA2_TRUE, RANGE_A_TRUE, NUGGET_TRUE, OZONE_MEAN, 21.0,
            ANISOTROPY_RATIO_TRUE
        )
        data_to_fit = torch.tensor(data_np, dtype=torch.float)[:N_SPATIAL_POINTS, :]
        z_data = data_to_fit[:, 0].numpy()
        z_centered_np = z_data - np.mean(z_data)
        z_centered_torch = torch.tensor(z_centered_np, dtype=torch.float)

        # --- B. Run L-BFGS ---
        raw_params_lbfgs = torch.tensor(
            initial_params_stable, 
            dtype=torch.float, 
            requires_grad=True
        )
        optimizer_lbfgs = optim.LBFGS(
            [raw_params_lbfgs], 
            lr=1.0, 
            max_iter=LBFGS_MAX_STEPS,
            max_eval=LBFGS_MAX_EVAL 
        )
        final_loss_lbfgs = torch.tensor(0.0)

        def closure_lbfgs():
            optimizer_lbfgs.zero_grad()
            loss = neg_log_likelihood_torch_stable(
                raw_params_lbfgs, d_lon_sq_torch, d_lat_sq_torch, z_centered_torch
            )
            if not torch.isinf(loss) and not torch.isnan(loss):
                loss.backward()
            return loss

        for step in range(LBFGS_MAX_STEPS):
            loss = optimizer_lbfgs.step(closure_lbfgs)
            final_loss_lbfgs = loss
            
            if PRINT_EPOCHS and ((step + 1) % 5 == 0): 
                # (Logging is silenced by default)
                pass 

        # --- C. Run Adam ---
        raw_params_adam = torch.tensor(
            initial_params_stable, 
            dtype=torch.float, 
            requires_grad=True
        )
        optimizer_adam = optim.Adam([raw_params_adam], lr=ADAM_LEARNING_RATE)
        final_loss_adam = torch.tensor(0.0)

        for epoch in range(ADAM_ITERATIONS):
            optimizer_adam.zero_grad()
            loss = neg_log_likelihood_torch_stable(
                raw_params_adam, d_lon_sq_torch, d_lat_sq_torch, z_centered_torch
            )
            if torch.isinf(loss) or torch.isnan(loss):
                continue 
            loss.backward()
            optimizer_adam.step()
            final_loss_adam = loss
            
            if PRINT_EPOCHS and ((epoch + 1) % 50 == 0): 
                # (Logging is silenced by default)
                pass
        
        # --- D. Store Results for this run ---
        
        # --- 💥 MODIFIED: L-BFGS Results (Log-Frame) 💥 ---
        with torch.no_grad():
            phi1_lbfgs = torch.exp(raw_params_lbfgs[0]).item()
            phi2_lbfgs = torch.exp(raw_params_lbfgs[1]).item()
            phi3_lbfgs = torch.exp(raw_params_lbfgs[2]).item()
            fitted_nugget_lbfgs = torch.exp(raw_params_lbfgs[3]).item()
        
        results_lbfgs.append({
            'sigma2': phi1_lbfgs / (phi2_lbfgs + 1e-6),
            'range_a': 1.0 / (phi2_lbfgs + 1e-6),
            'ratio': np.sqrt(phi3_lbfgs),
            'nugget': fitted_nugget_lbfgs,
            'nll': final_loss_lbfgs.item()
        })
        
        # --- 💥 MODIFIED: Adam Results (Log-Frame) 💥 ---
        with torch.no_grad():
            phi1_adam = torch.exp(raw_params_adam[0]).item()
            phi2_adam = torch.exp(raw_params_adam[1]).item()
            phi3_adam = torch.exp(raw_params_adam[2]).item()
            fitted_nugget_adam = torch.exp(raw_params_adam[3]).item()
        
        results_adam.append({
            'sigma2': phi1_adam / (phi2_adam + 1e-6),
            'range_a': 1.0 / (phi2_adam + 1e-6),
            'ratio': np.sqrt(phi3_adam),
            'nugget': fitted_nugget_adam,
            'nll': final_loss_adam.item()
        })
        
        run_end_time = time.time()
        print(f"--- Simulation {i+1} complete. Time elapsed: {run_end_time - run_start_time:.2f}s ---")


    print("\n--- All Simulations Complete ---")
    end_time_total = time.time()

    # ==========================================================
    # --- 4. Display Aggregate Results ---
    # ==========================================================
    
    print("\n" + "="*75)
    print(f"TARGET PARAMETERS: Variance (σ²)={SIGMA2_TRUE}, Range (a)={RANGE_A_TRUE}, Anisotropy (θ₃-ratio)={ANISOTROPY_RATIO_TRUE}, Nugget (η²)={NUGGET_TRUE}")
    print(f"AGGREGATE RESULTS OVER {NUM_SIMULATIONS} RUNS")
    print(f"Grid Size: {GRID_X}x{GRID_Y} (N = {N_SPATIAL_POINTS})")
    print(f"Total time: {end_time_total - start_time_total:.2f} seconds")
    print("="*75)

    # Helper function to calculate and print stats
    def print_stats(results_list, optimizer_name):
        # Convert list of dicts to a dict of numpy arrays
        # Filter out NaN/Inf values that could arise from failed runs
        params = {}
        for key in results_list[0].keys():
            valid_values = [res[key] for res in results_list if np.isfinite(res[key])]
            if not valid_values:
                valid_values = [np.nan] # Handle case where all runs failed for a param
            params[key] = np.array(valid_values)
        
        num_valid = len(params['nll'])
        num_total = len(results_list)

        print(f"✨ {optimizer_name} Average Results ({num_valid}/{num_total} Valid Runs):")
        
        # Helper for printing mean/std
        def ms(key):
            return f"{np.mean(params[key]):.3f} (Std: {np.std(params[key]):.3f})"

        print(f"  * Fitted Variance (σ²): {ms('sigma2')} (Target: {SIGMA2_TRUE})")
        print(f"  * Fitted Range (a): {ms('range_a')} (Target: {RANGE_A_TRUE})")
        print(f"  * Fitted Anisotropy (θ₃-ratio): {ms('ratio')} (Target: {ANISOTROPY_RATIO_TRUE})")
        print(f"  * Fitted Nugget (η²): {ms('nugget')} (Target: {NUGGET_TRUE})")
        print(f"  * Final -LL Value: {np.mean(params['nll']):.2f} (Std: {np.std(params['nll']):.2f})")

    try:
        if results_lbfgs:
            print_stats(results_lbfgs, "PyTorch L-BFGS")
        else:
            print("No valid L-BFGS results to display.")
            
        print("\n" + "-"*75 + "\n") # Add a separator
        
        if results_adam:
            print_stats(results_adam, "PyTorch Adam")
        else:
            print("No valid Adam results to display.")

    except Exception as e:
        print(f"An error occurred during final reporting: {e}")
        print("\nRaw L-BFGS Results:", results_lbfgs)
        print("\nRaw Adam Results:", results_adam)

    print("="*75)

--- Starting 1 Simulation Runs ---
--- Grid Size: 80x56 (N = 4480) ---
--- Running Simulation 1/1 ---
--- Simulation 1 complete. Time elapsed: 252.11s ---

--- All Simulations Complete ---

TARGET PARAMETERS: Variance (σ²)=11.0, Range (a)=1.9, Anisotropy (θ₃-ratio)=0.5, Nugget (η²)=0.3
AGGREGATE RESULTS OVER 1 RUNS
Grid Size: 80x56 (N = 4480)
Total time: 252.26 seconds
✨ PyTorch L-BFGS Average Results (1/1 Valid Runs):
  * Fitted Variance (σ²): 7.925 (Std: 0.000) (Target: 11.0)
  * Fitted Range (a): 1.326 (Std: 0.000) (Target: 1.9)
  * Fitted Anisotropy (θ₃-ratio): 0.476 (Std: 0.000) (Target: 0.5)
  * Fitted Nugget (η²): 0.328 (Std: 0.000) (Target: 0.3)
  * Final -LL Value: 2919.92 (Std: 0.00)

---------------------------------------------------------------------------

✨ PyTorch Adam Average Results (1/1 Valid Runs):
  * Fitted Variance (σ²): 8.071 (Std: 0.000) (Target: 11.0)
  * Fitted Range (a): 1.241 (Std: 0.000) (Target: 1.9)
  * Fitted Anisotropy (θ₃-ratio): 0.594 (Std: 0.000) (T